# Testing the action potential of cell models by using standard parameters


In [33]:
# fileName

fileName = 'eg_testing-cell-models'

In [34]:
# libs

import pyNN.brian2 as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt


In [35]:
# simulation settings

sim.setup(
        timestep=1, # [ms]
        min_delay=1, # [ms]
        max_delay=100) # [ms]

simtime = 250 # [ms+]



## make the network

In [36]:
# define make_pop

pops = {}

def make_pop(cell_model):
    pops = sim.Population(
                            1, # one cell in each cell model
                            cell_model,
                            #cellparams=None, # std pars used
                            structure = space.Line(dx=1.0, x0=0.0, y=0.0, z=0.0),
                            #initial_values=None,
                            #label=str(cell_model),
                            #constraints=None,
                            #additional_parameters=None,
                            )
    pops.record('v')
    
    return pops




In [37]:
# compute pops

for idx, value in enumerate([
                            sim.IF_cond_exp(), 
                            sim.IF_curr_alpha(),
                            sim.IF_curr_exp(),
                            sim.IF_cond_exp_gsfa_grr(),
                            sim.EIF_cond_alpha_isfa_ista(), 
                            sim.EIF_cond_exp_isfa_ista(),
                            sim.Izhikevich()]):
                            #sim.extra_models.IF_curr_dual_exp,
                            #sim.extra_models.IFCondExpStoc,
                            #sim.extra_models.IFCurDelta,
                            #sim.extra_models.IFCurrExpCa2Adaptive,
                            #sim.extra_models.Izhikevich_cond]):

    pops['model_' + str(idx)] = make_pop(value) 

In [38]:
# check pops

pops.keys()

dict_keys(['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6'])

## make the thalamic input stimulus

In [39]:
# a sequence of ten spikes with regularly spaced spike times

pops['thalamus'] = sim.Population(
                            1, # one thalamic cell for each cell model
                            sim.SpikeSourceArray(spike_times=np.arange(50,60)), # ten sequential spike times
                            structure = space.Line(dx=1.0, x0=0.0, y=0.0, z=0.0),
                            )    

pops.keys()

dict_keys(['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'thalamus'])

## make the projections

In [40]:
projs = {}

for i in range(len(pops.keys())-1):
    projs['thalamus', 'model_' + str(i)] = sim.Projection(
                                    pops['thalamus'],
                                    pops['model_' + str(i)],
                                    sim.OneToOneConnector(),
                                    synapse_type=sim.StaticSynapse(weight=0.1, delay=1.0),
                                    receptor_type = 'excitatory',
                                    space = space.Space(axes = 'x'),
                                    label=None,
                                )

projs.keys()

dict_keys([('thalamus', 'model_0'), ('thalamus', 'model_1'), ('thalamus', 'model_2'), ('thalamus', 'model_3'), ('thalamus', 'model_4'), ('thalamus', 'model_5'), ('thalamus', 'model_6')])

## run the simulation

In [41]:
sim.run(simtime) 

INFO       No numerical integration method specified for group 'neurongroup_13', using method 'euler' (took 0.02s, trying other methods took 0.11s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup_14', using method 'exact' (took 14.73s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup_15', using method 'exact' (took 6.24s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup_19', using method 'euler' (took 0.03s, trying other methods took 0.19s). [brian2.stateupdaters.base.method_choice]


BrianObjectException: Error encountered with object named 'neurongroup_16_spike_thresholder'.
Object was created here (most recent call only, full details in debug log):
  File '/home/alberto-int/anaconda3/lib/python3.7/site-packages/pyNN/brian2/cells.py', line 69, in __init__
    clock=simulator.state.network.clock)

An error occurred when preparing an object. (See above for original error message and traceback.)

## save results

In [ ]:
# save the results

outputs = {}

for i in range(len(pops.keys())-1):
    outputs['model_' + str(i)] = pops['model_' + str(i)].get_data()
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops['model_' + str(i)].write_data(fileName + '_' + str(recording) + '.pkl')


## recover results

In [ ]:
# make the recover results function

def recover_results(outputs):
    results = {}
    for key in outputs.keys(): 
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results


In [ ]:
# recover results

results = recover_results(outputs)
results.keys()

## check the voltage signature

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(3, 3, sharex=True, sharey=False, figsize=(11,7))
fig.tight_layout(pad=3)
fig.suptitle('voltage signature in cell models')

axes_list = fig.axes
idx = 0
for i in range(len(pops.keys())-1):
    im = axes_list[i].plot(results['model_' + str(i), 'v'], label=str('model_' + str(i)))
    #axes_list[i].set_xticklabels([])
    axes_list[i].legend()
    axes_list[i].grid(which='both')
    if i==0 or i==3 or i==6:
        axes_list[i].set_ylabel('[mv]')
    if i==6 or i==7 or i==8:
        axes_list[i].set_xlabel('[ms]')


In [14]:
# end simulation

sim.end()

## Task 1: on the action potential of cell models

- Have a look of the plot, you should observe the different voltage signatures having similarities as well as differences. <br>
  Describe them or try making a classification by adopting criteria indicated in this bottom image<br>
  
  e.g., 
      - depolarization timing
      - shape of depolarization
      - action potential timing
      - shape of repolarization
      - presence/absence of refractory period
      - resting state timing
      
      
- This simulation has been done with standard parameters. <br>
  Choose a parameter shared from all the models and tune it. <br>
  Observe how the voltage trace of the action potential changes. <br>
  To explore the default paramters, use e.g., `sim.IF_cond_exp.default_parameters` <br>
  Change them by defining a new dictionary of pars for each model call e.g., pars[model_x]<br>
  

![](https://upload.wikimedia.org/wikipedia/commons/4/4a/Action_potential.svg)

## Task 2: on the network of different cell models

- The previous results are related to single behaviour of the given cell models.<br>
  Try to make a population of 25 cells for each model and record their voltage.<br>
  The input should be design as a sequence of ten input spikes separeted 150 ms for each model.<br>
  E.g., the first model receives input at 50-59ms, the second at 200-209ms, the third at 350-359ms etc.<br>
  Plot the results in a single plot showing the all traces that should appear sequentally displayed.<br>
  Set the simtime at 1500ms.